<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/GPT_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import json
import re
import string
from IPython.display import display , HTML
import tensorflow as tf
from tensorflow.keras import layers, models, losses , callbacks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#0. Parameters

In [4]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

#1. Load the Data


In [5]:
#Load the full dataset
with open("/app/data/wine-reviews/winemag-data-130k-v2.json") as json_data:
  wine_data = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: '/app/data/wine-reviews/winemag-data-130k-v2.json'

In [6]:
wine_data[10]

NameError: name 'wine_data' is not defined

In [7]:
#filter the dataset
filtered_data = [
    "wine review: "
    + x["country"]
    + ":"
    + x["province"]
    + ":"
    + x["variety"]
    + " :"
    + x["description"]
    for x in wine_data
    if x["country"] is not None
    and x["province"] is not None
    and x["variety"] is not None
    and x["description"] is not None

]

NameError: name 'wine_data' is not defined

In [8]:
#Count the recipes
n_wines = len(filtered_data)
print(f"{n_wines} recipes loaded")

NameError: name 'filtered_data' is not defined

In [9]:
example = filtered_data[25]
print(example)

NameError: name 'filtered_data' is not defined

#2. Tokenize the data

In [10]:
#pad the punctuation , to treat them as seperate 'words'
def pad_punctuation(s):
  s = re.sub(f"([{string.punctuation},  '\n'])" , r"\1" , s)
  s = re.sub("+" , " ", s)
  return s

text_data = [pad_punctuation(x) for x in filtered_data]

NameError: name 'filtered_data' is not defined

In [11]:
#Display an example of a recipe
example_data = text_data[25]
example_data

NameError: name 'text_data' is not defined

In [12]:
#Convert to a tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensorflow_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

AttributeError: type object 'DatasetV2' has no attribute 'from_tensorflow_slices'

In [13]:
#Create a vectorization layer
vectorize_layer = layers.TextVectorization(
    standardize = "lower",
    max_tokens = VOCAB_SIZE ,
    output_mode = "int",
    output_sequence_length = MAX_LEN + 1
)

In [14]:
#Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

NameError: name 'text_ds' is not defined

In [15]:
#Display some token:word mappings
for i , word in enumerate(vocab[:10]):
  print(f"{i}: {word}")

NameError: name 'vocab' is not defined

In [16]:
#Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

NameError: name 'example_data' is not defined

#3. Create the training Set

In [17]:
#create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
  text = tf.expand_dims(text, -1)
  tokenized_sentences = vectorize_layer(text)
  x = tokenized_sentences[:,: -1]
  y = tokenized_sentences[:,1:]
  return x , y

train_ds = text_ds.map(prepare_inputs)

NameError: name 'text_ds' is not defined

In [18]:
example_input_output = train_ds.take(1).get_single_element()

NameError: name 'train_ds' is not defined

In [19]:
#example Input
example_input_output[0][0]

NameError: name 'example_input_output' is not defined

In [20]:
#Example Output (Shifted by one token)
example_input_output[1][0]

NameError: name 'example_input_output' is not defined

#Create the casual attention mask function

In [23]:
def causal_attention_mask(batch_size , n_dest , n_src , dtype):
  i = tf.range(n_dest)[:, None]
  j = tf.range(n_src)
  m = i >= j - n_src + n_dest
  mask = tf.cast(m, dtype)
  mask = tf.reshape(mask, [1, n_dest, n_src])
  mult = tf.concat(
      [tf.expand_dims(batch_size , -1), tf.constant([1,1] , dtype = tf.int32)], 0
  )
  return tf.tile(mask, mult)

np.transpose(causal_attention_mask(1,10, 10, dtype = tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

#6. Create a Transformer Block layer

In [24]:
class Transformer(layers.Layer):
  def __init__(self, num_heads, key_dim , embed_dim , ff_dim , dropout_rate = 0.1):
    super(TransformerBlock, self).__init__()
    self.num_heads = num_heads
    self.key_dim = key_dim
    self.embed_dim = embed_dim
    self.ff_dim = ff_dim
    self.dropout_rate = dropout_rate
    self.attn = layers.MultiHeadAttention(
        num_heads, key_dim , output_shape = embed_dim
    )
    self.dropout_1 = layers.Dropout(self.dropout_rate)
    self.ln_1 = layers.LayerNormalization(epsilon = 1e-6)
    self.ffn_1 = layers.Dense(self.ff_dim, activation = "relu")
    self.ffn_2 = layers.Dense(self.embed_dim)
    self.dropout_2 = layers.Dropout(self.dropout_rate)
    self.ln_2 = layers.LayerNormalization(epsilon = 1e-6)

  def call(self, inputs ):
    input_shape = tf.shape(inputs)
    batch_size = input_shape[0]
    seq_len = input_shape[1]
    casual_mask = casual_attention_mask(
        batch_size , seq_len, seq_len , tf.bool
    )
    attention_output , attention_scores = self.attn(
        inputs ,
        inputs ,
        attention_mask = casual_mask,
        return_attention_scores = True,

    )
    attention_output = self.dropout_1(attention_output)
    out1 = self.ln_1(inputs + attention_output)
    ffn_1 = self.ffn_1(out1)
    ffn_2 = self.ffn_2(ffn_1)
    ffn_output = self.dropout_2(ffn_2)
    return (self.ln_2(out1 + ffn_output) , attention_scores)

  def get_config(self):
    config = super().get_config()
    config.update(
        {
            "key_dim": self.key_dim,
            "embed_dim": self.embed_dim,
            "num_heads":self.num_heads,
            "ff_dim" :self.ff_dim,
            "dropout_rate": self.dropout_rate,

        }
    )
    return config

#7. Create the token and Position Embedding


In [1]:
class TokenAndPositionEmbedding(layers.Layer):
  def __init__(self, max_len, vocab_size , embed_dim):
    super(TokenAndPositionEmbedding, self).__init()
    self.max_len = max_len
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim
    self.token_emb = layers.Embedding(
        input_dim = vocab_size , output_dim = embed_dim
    )
    self.pos_emb = layers.Embeddin(input_dim = max_len, output_dim = embed_dim)

  def call(self, x):
    maxlen = tf.shape(x)[-1]
    positions = tf.range(start = 0, limit = maxlen, delta = 1)
    positions = self.pos_emb(position)
    x = self.token_emb(x)
    return x + positions

  def get_config(self):
    config = super().get_config()
    config.update(
        {
            "max_len": self.max_len,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        }
    )
    return config

NameError: name 'layers' is not defined

#8. Bulid the Transforer model


In [2]:
inputs = layers.Inupt(shape= (None) , dtype = tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x , attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM , EMBEDDING_DIM , FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE , activation = "softmax")(x)
gpt = models.Model(inputs  = inputs , outputs = [outputs , attention_scores])
gpt.compile("adam" , loss = [losses.SparseCategoricalCrossentropy(), None])



NameError: name 'layers' is not defined

In [3]:
gpt.summary()

NameError: name 'gpt' is not defined

In [4]:
if LOAD_MODEL:
  #model.load_weights('./models/model')
  gpt = models.load_model("./models/gpt" , compile = True)

NameError: name 'LOAD_MODEL' is not defined

#9. Train the Transformer

In [5]:
#create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
  def __init__(self, index_to_word , top_k = 10):
    self.index_to_word = index_to_word
    self.word_to_index = {
        word: index for index, word in enumerate(index_to_word)
    }
  def sample_from(self, probs, temperature):
    probs = probs ** (1/ temperature)
    probs = probs / np.sum(probs)
    return np.random.choice(len(probs) , p = probs), probs

  def generate(self, start_prompt , max_tokens , temperature):
    start_tokens = [
        self.word_to_index.get(x, 1) for x in start_prompt.split()
    ]
    sample_token = None
    info = []
    while len(start_tokens) < max_tokens and sample_token != 0:
      x = np.array([start_tokens])
      y , att = self.model.predict(x, verbose = 0)
      sample_token, probs = self.sample_from(y[0][-1] , temperature)
      info.append(
          {
              "prompt" : start_prompt,
              "word_probs": probs,
              "atts":att[0, : , -1, :],

          }

      )
      start_tokens.append(sample_token)
      start_prompt = start_prompt + " " + self.index_to_word[sample_token]
    print(f"\ngenerated text:\n{start_prompt}\n")
    return info

  def on_epoch_end(self, epoch , logs = None):
    self.generate("wine review" , max_tokens = 80 , temperature = 1.0)

NameError: name 'callbacks' is not defined

In [6]:
#Create a model save a checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint/checkpoint.ckpt",
    save_weights_only = True,
    save_freq = "epoch",
    verbose = 0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

#Tokenize starting prompt
text_generator = TextGenerator(vocab)

NameError: name 'callbacks' is not defined

In [7]:
gpt.fit(
    train_ds,
    epochs = EPOCHS ,
    callbacls = [model_checkpoint_callback, tensorboard_callback , text_generator],
)

NameError: name 'gpt' is not defined

In [8]:
#save the final model
gpt.save("./models/gpt")

NameError: name 'gpt' is not defined

#3. Generate text using the Transformer

In [9]:
def print_probs(info , vocab , top_k = 5):
  for i in info :
    higlighted_text = []
    for word , att_score in zip(
        i["prompt"].split() , np.mean(i["atts"] , axis = 0)
    ):
        highlightes_text.append(
            '<span style= "background-color:rgba(135, 206, 250,
            + str(att_score / max(np.mean(i["atts"] , axis = 0)))
            +');">'
            + word
            +"</span>"
            )
    highlighted_text = " " . join(highlighted_text)
    display(HTML(highlighted_text))

    word_probs = i["word_probs"]
    p_sorted  = np.sort(word_probs)[::-1][:top_k]
    i_sorted = np.argsort(word_probs)[:: -1][:top_k]
    for p , in zip(p_sorted, i_sorted):
      print(f"{vocab[i]}: \t{np.round(100* p , 2)}%")
    print("-------\n")

SyntaxError: unterminated string literal (detected at line 8) (<ipython-input-9-ef21e9528904>, line 8)

In [10]:
info = text_generator.generate(
    "wine review: us" , max_tokens = 80 , temperature - 1.0
)

SyntaxError: positional argument follows keyword argument (<ipython-input-10-71b968faa03a>, line 3)

In [11]:
info = text_generator.generate(
    "wine review :italy", max_tokens = 80, temperature = 0.5
)


NameError: name 'text_generator' is not defined

In [ ]:
info = text_generator.generate(
    "wine review: germany", max_tokens = 80, temperature = 0.5
)
print_probs(info , vocab)